In [1]:
import pandas as pd
import numpy as np

### data load

In [3]:
# path 
import os
os.chdir('/home/midan/_midanniiii/IMGTHLA/fasta')

In [4]:
import Bio

In [5]:
# for HLA-B
from Bio import SeqIO, Seq
input_file = 'B_prot.fasta'
records = SeqIO.parse(input_file, 'fasta')
records = list(records) # make a copy, otherwise our generator is exhausted after calculating maxlen
maxlen = max(len(record.seq) for record in records)

# pad sequences so that they all have the same length
for record in records:
    if len(record.seq) != maxlen:
        sequence = str(record.seq).ljust(maxlen, '.')
        record.seq = Seq.Seq(sequence)
assert all(len(record.seq) == maxlen for record in records)

In [6]:
des = []
for record in records:
    des.append(record.description)

In [7]:
des[0]

'HLA:HLA00132 B*07:02:01:01 362 bp'

In [8]:
des[200].count(':')

2

In [11]:
import pandas as pd

In [12]:
ids = []
full_ids = []
delete = []
name = []
for i in range(len(records)):
    des = records[i].description[13:] # A*01:01:01:01 365 bp
    if des[0] == 'B':
        full_ids.append(des[2:des.find('bp')-5])
        name.append(records[i].name)
        if des.count(':')<2:
            ids.append(des[2:des.find('bp')-4])
        else:
            ids.append(des[2:des.find(':', des.find(':')+1)])

In [13]:
df = pd.DataFrame(data = {'ids': ids, 'full_id': full_ids, 'name':name})

In [14]:
df

,ids,full_id,name
0,07:02,07:02:01:01,HLA:HLA00132
1,07:02,07:02:01:02,HLA:HLA16169
2,07:02,07:02:01:03,HLA:HLA16634
3,07:02,07:02:01:04,HLA:HLA16855
4,07:02,07:02:01:05,HLA:HLA17044
...,...,...,...
7119,82:02,82:02:01:01,HLA:HLA01188
7120,82:02,82:02:01:02,HLA:HLA18126
7121,82:02,82:02:02,HLA:HLA11935
7122,82:03,82:03,HLA:HLA04447


### delete overlapped

In [15]:
uniques = list(df['ids'].unique())

In [16]:
len(uniques)

4840

In [17]:
overlap = [] # alleles of overlapped
not_overlap = [] # alleles of not overlapped
for uniq in uniques:
    if sum(df.ids == uniq) != 1:
        overlap.append(uniq) 
    else:
        not_overlap.append(uniq)

In [18]:
len(not_overlap)

4519

In [19]:
only = [] # delete overlapped alleles
for ovl in overlap:
    ovls = []
    for i in range(len(df)):
        if df.ids[i] == ovl:
            ovls.append(i)
        elif df.ids[i] in not_overlap:
            only.append(i)
    only.append(ovls[0])
only = sorted(list(set(only)))

In [20]:
len(only)

4840

In [21]:
import numpy as np

In [22]:
data = np.asarray(df)[only]

In [23]:
hla_a = []
a_name = []
for i in range(len(data)):
    hla_a.append('HLA-B-' + data[i][0][0:2] + data[i][0][3:5])
    a_name.append(data[i][2])

In [24]:
pd.DataFrame(data={'alleles':hla_a, 'name':a_name})

,alleles,name
0,HLA-B-0702,HLA:HLA00132
1,HLA-B-0703,HLA:HLA00135
2,HLA-B-0704,HLA:HLA00136
3,HLA-B-0705,HLA:HLA00137
4,HLA-B-0706,HLA:HLA00138
...,...,...
4835,HLA-B-8109,HLA:HLA26055
4836,HLA-B-8201,HLA:HLA00399
4837,HLA-B-8202,HLA:HLA01188
4838,HLA-B-8203,HLA:HLA04447


In [27]:
from Bio import Align
from Bio.Align import Applications
from Bio.Align.Applications import MuscleCommandline

In [25]:
from Bio import SeqIO
records = (r for r in SeqIO.parse('B_prot.fasta', 'fasta') if len(r)<900)

In [26]:
from io import StringIO
handle = StringIO()
SeqIO.write(records, handle, 'fasta')
data = handle.getvalue()

In [28]:
muscle_exe = '/home/midan/_midanniiii/muscle3.8.31_i86linux64'
muscle_cline = MuscleCommandline(muscle_exe)
stdout, stderr = muscle_cline(stdin=data)

In [29]:
empty = []
new = []
for i in range(len(stdout)):
    if stdout[i] == '>':
        new.append(empty)
        empty = []
    else:
        empty.append(stdout[i])

In [30]:
align = []
for i in range(len(new)):
    align.append((','.join(new[i])).replace(',',''))

In [31]:
# alignment & name
new_align = []
names = []
for i in range(len(align)):
    a = align[i]
    p = a.find('p')
    new_align.append(align[i][p+2:])
    names.append(align[i][:12])

In [32]:
names

['',
 'HLA:HLA01187',
 'HLA:HLA22365',
 'HLA:HLA21396',
 'HLA:HLA01532',
 'HLA:HLA00163',
 'HLA:HLA10725',
 'HLA:HLA20248',
 'HLA:HLA21177',
 'HLA:HLA24752',
 'HLA:HLA04458',
 'HLA:HLA22535',
 'HLA:HLA10695',
 'HLA:HLA05284',
 'HLA:HLA07189',
 'HLA:HLA16997',
 'HLA:HLA02315',
 'HLA:HLA06135',
 'HLA:HLA03263',
 'HLA:HLA11419',
 'HLA:HLA06185',
 'HLA:HLA08506',
 'HLA:HLA15167',
 'HLA:HLA07303',
 'HLA:HLA02783',
 'HLA:HLA00216',
 'HLA:HLA16757',
 'HLA:HLA10403',
 'HLA:HLA08936',
 'HLA:HLA06213',
 'HLA:HLA03946',
 'HLA:HLA13652',
 'HLA:HLA19497',
 'HLA:HLA01782',
 'HLA:HLA03462',
 'HLA:HLA06214',
 'HLA:HLA20313',
 'HLA:HLA01508',
 'HLA:HLA22600',
 'HLA:HLA22782',
 'HLA:HLA09077',
 'HLA:HLA03956',
 'HLA:HLA11230',
 'HLA:HLA05125',
 'HLA:HLA06232',
 'HLA:HLA01177',
 'HLA:HLA10908',
 'HLA:HLA06575',
 'HLA:HLA02851',
 'HLA:HLA02995',
 'HLA:HLA07906',
 'HLA:HLA02388',
 'HLA:HLA07632',
 'HLA:HLA19331',
 'HLA:HLA12460',
 'HLA:HLA19164',
 'HLA:HLA04752',
 'HLA:HLA15546',
 'HLA:HLA02698',
 'HLA:HLA

In [33]:
for i in range(len(new_align)):
    new_align[i] = new_align[i].replace('\n','')
new_align = new_align[1:]
names = names[1:]

In [34]:
m = new_align[0].find('M')
for i in range(len(new_align)):
    new_align[i] = new_align[i][m:]

In [35]:
new_align

['MRSRRP-----EP------------------------------------------S---------------------------------------------SCC-SG--------------------------GQWPX---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------',
 'PARPRG-----APLHLS-GL----------------------------------RG------------------RHAVREVRQ-RRRESERGAAGAVDRAGGAGILGPE--------------------HTDLQD--------------------------QHTDX-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------',
 'SGPDRD-----------LGRLPLHEVFLHRHVPARPRGAP----LHHRGL----RG------------------RHAVREVRQ-RRHESEEGAAGAMDRAGGAGVLGPGDTDLQDQHT-------------DLPR-EPAHRAPL---LQPERGRVSHHPE-DVR---------------LRR-------GAGRAPPPRVX--------------------------------------------------------------

In [36]:
df_a = pd.DataFrame(data = {'align' : new_align, 'name':names})
a = pd.DataFrame(data = {'name': a_name, 'alleles': hla_a})
hla_a = pd.merge(a,df_a)
hla_b = hla_a.drop(['name'], axis=1)

In [37]:
hla_b

,alleles,align
0,HLA-B-0702,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...
1,HLA-B-0703,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...
2,HLA-B-0704,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...
3,HLA-B-0705,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...
4,HLA-B-0706,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...
...,...,...
4834,HLA-B-8109,SRPGRG-----EPRFISMGYVDDTQFVRFDSDAASPREEPRAPWIE...
4835,HLA-B-8201,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...
4836,HLA-B-8202,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...
4837,HLA-B-8203,SRPGRG-----EPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIE...


In [38]:
aligns = list(hla_b['align'])
align = aligns[0]
indices = []
for i in range(len(align)):
    if align[i] =='-':
        indices.append(i)

In [39]:
new_align = []
for i in range(len(aligns)):
    align = aligns[i]
    a = []
    for j in range(len(align)):
        if j not in indices:
            a.append(align[j])
    new_align.append(('').join(a))
    new_align[i] = new_align[i].replace('-','*')

In [40]:
new_align

['SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPEYWDRNTQIYKAQAQTDRESLRNLRGYYNQSEAGSHTLQSMYGCDVGPDGRLLRGHDQYAYDGKDYIALNEDLRSWTAADTAAQITQRKWEAAREAEQRRAYLEGECVEWLRRYLENGKDKLERADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAACSDSAQGSDVSLTA',
 'SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPEYWDRNTQIYKTNTQTDRESLRNLRGYYNQSEAGSHTLQSMYGCDVGPDGRLLRGHDQYAYDGKDYIALNEDLRSWTAADTAAQITQRKWEAAREAEQRRAYLEGECVEWLRRYLENGKDKLERADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAACSDSAQGSDVSLTA',
 'SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPEYWDRNTQIYKAQAQTDRESLRNLRGYYNQSEAGSHTLQSMYGCDVGPDGRLLRGHDQYAYDGKDYIALNEDLRSWTAADTAAQITQRKWEAAREAEQDRAYLEGECVEWLRRYLENGKDKLERADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAACSDSAQGSDVSLTA',
 'SRPGR

In [41]:
hla_b = pd.DataFrame(data = {'alleles':list(hla_b['alleles']), 'align': new_align})

In [42]:
hla_b

,alleles,align
0,HLA-B-0702,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
1,HLA-B-0703,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
2,HLA-B-0704,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
3,HLA-B-0705,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
4,HLA-B-0706,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
...,...,...
4834,HLA-B-8109,SRPGRGEPRFISMGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
4835,HLA-B-8201,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
4836,HLA-B-8202,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...
4837,HLA-B-8203,SRPGRGEPRFISVGYVDDTQFVRFDSDAASPREEPRAPWIEQEGPE...


In [43]:
hla_b.to_csv('/home/midan/_midanniiii/HLA_B_prot.txt', index=False, header=None, sep='\t')